# Multi-scenario
Multiple scenarios can be defined in a CSV file with rows for each scenrario and columns for the inputs. `all_api_inputs.csv` provides a template for all of the possible header values (input keys).

Let's take a look at the example `scenarios.csv`:

In [7]:
import json  # needed to load in our scenario
import os  # to handle file paths across operating systems
import pandas as pd  # only used to show the csv file
from src.post_and_poll import get_api_results
API_KEY = 'DEMO_KEY'  # REPLACE WITH YOUR API KEY

In [8]:
# following is not necessary but silences warnings:
# InsecureRequestWarning: Unverified HTTPS request is being made to host 'developer.nrel.gov'. Adding certificate verification is strongly advised.
import urllib3
urllib3.disable_warnings()

In [9]:
df = pd.read_csv(os.path.join("inputs", "scenarios.csv"))

df.head()

,site_number,description,latitude,longitude,PV|max_kw,urdb_label,load_file,Wind|max_kw
0,1,site3-No-PV,34.1404,-118.2764,0.0,5a3821035457a32645d2dd80,site3_8760_loads_kw.csv,0
1,2,site3-With-PV,34.1404,-118.2764,NaN,5a3821035457a32645d2dd80,site3_8760_loads_kw.csv,0


We have two Scenarios:
1. One with no PV nor Wind evaluated (by setting their `max_kw`s to zero)
2. One with PV and no Wind evaluated

Both Scenarios have the same location, the same electricity tariff (set via the `urdb_label`), and use the same custom load profile - which is passed in via another csv file.

In [10]:
"""
Here are some convenience definitions for using the Multi-scenario capabilities
"""
##############################################################################################################
inputs_path = os.path.join(".", 'inputs')
outputs_path = os.path.join(".", 'outputs')
output_template = os.path.join(outputs_path, 'results_template.csv')
output_file = os.path.join(outputs_path, 'results_summary.csv')
output_file_spreadsheet_template = os.path.join(outputs_path, 'template.xlsx')
output_file_spreadsheet = os.path.join(outputs_path, 'results_summary.xlsx')
##############################################################################################################

Let's start by converting the `scenarios.csv` into a list of POST's that we can send to the API:

In [11]:
from src.multi_site_inputs_parser import multi_site_csv_parser

path_to_inputs = os.path.join(inputs_path, 'scenarios.csv')

list_of_posts = multi_site_csv_parser(
    path_to_inputs, 
    api_url='https://developer.nrel.gov/api/reopt/v1', 
    API_KEY=API_KEY
)

Now we can collect all of the results using the `get_api_results` function from the Single Scenario example:

In [12]:
responses = []

for post in list_of_posts:
    responses.append(
        get_api_results(
            post, 
            results_file=os.path.join(outputs_path, post['Scenario']['description'] + '.json'),
            api_url="https://developer.nrel.gov/api/reopt/v1", 
            API_KEY=API_KEY
        )
    )

main         INFO     Response OK from https://developer.nrel.gov/api/reopt/v1/job/?api_key=uiMp56xYMT983T2OikI0RrEKAyAAEDpZHzuC8nTC.
main         INFO     Polling https://developer.nrel.gov/api/reopt/v1/job/d0e630b9-4f7e-4ac3-8f01-a42016fcaf12/results/?api_key=uiMp56xYMT983T2OikI0RrEKAyAAEDpZHzuC8nTC for results with interval of 5s...
main         INFO     Saved results to ./outputs/site3-No-PV.json
main         INFO     Response OK from https://developer.nrel.gov/api/reopt/v1/job/?api_key=uiMp56xYMT983T2OikI0RrEKAyAAEDpZHzuC8nTC.
main         INFO     Polling https://developer.nrel.gov/api/reopt/v1/job/dff2ba4d-4ef6-4dbd-b4c8-0a4c1d42682b/results/?api_key=uiMp56xYMT983T2OikI0RrEKAyAAEDpZHzuC8nTC for results with interval of 5s...
main         INFO     Saved results to ./outputs/site3-With-PV.json


Note that we used the `Scenario.description` to define the results file name. The `Scenario.description` was defined in the `scenarios.csv`.

### Summarizing multiple scenario results

There are two options for making a summary of multiple scenarios' resutls:
1. Write to a csv using a template with column headers for desired summary keys (scalar values only)
2. Write all inputs, outputs, and dispatch to an Excel spreadsheet


#### Option 1: Use a template CSV to collect certain results

In [7]:
from src.parse_api_responses_to_csv import parse_responses_to_csv_with_template

parse_responses_to_csv_with_template(
    csv_template=output_template, 
    responses=responses, 
    output_csv=output_file, 
    input_csv=path_to_inputs,
    n_custom_columns=2
)

#### Option 2: Write all results out to an Excel file

In [8]:
os.getcwd()

'/home/jovyan/work'

In [9]:
from src.parse_api_responses_to_excel import parse_api_responses_to_excel

parse_api_responses_to_excel(responses, template=output_file_spreadsheet_template, spreadsheet=output_file_spreadsheet)

FileNotFoundError: [Errno 2] No such file or directory: './outputs/template.xlsx'